# Installing Relevant Libraries To Store the Data

In [ ]:
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6   #egg=detectron2 -q
!apt-get install poppler-utils
!pip install pinecone-client
!pip install tiktoken
#!pip install pillow==8.3.2
!pip install pillow
!pip install openai==0.28.0
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━

# Importing Libraries

In [ ]:
import os
import openai
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import SentenceTransformerEmbeddings
import pandas as pd
from pinecone import Pinecone

# Directory Loader Function Take the Data From Directory to Store Data There

In [ ]:
#### directory = ''
'
'''
The purpose of this function is to store data
'''
def load_docs():
    # Get user input for the directory
    directory = input("Enter the directory where the PDF documents are stored: ")
    # Use the provided directory in the loader
    loader = DirectoryLoader(directory, glob='**/*.pdf')
    #Loader.Load Function Load the the in pdf_documents Variable
    pdf_documents = loader.load()
    # Return Pdf Files
    return pdf_documents

#This Functions is taking Pdf_docs
pdf_documents = load_docs()


Enter the directory where the PDF documents are stored: /content/drive/MyDrive/content


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


#### Printing How Many Documents Are There

In [ ]:
#Checking the Numeber of Documents
print('Checking How Many Number Of Documents Are There', len(pdf_documents))

Checking How Many Number Of Documents Are There 19


# Splitting Docs Into Chunks Due to Token Limit

In [ ]:
'''
The Purpose Of This Function is to Split Docs.

'''
def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  #Text Splitter Function Split the Data Into Chunks
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  #Splitting Documents Using Split Documents Function
  docs = text_splitter.split_documents(documents)
  #Return Docs
  return docs
#Calling Split_Docs Function to Get Chunks
Chunks = split_docs(pdf_documents)
#Printing the total Number of Chunks
print('The Total Number of Chunks Are',len(Chunks))

The Total Number of Chunks Are 1321


# Intialize Embeddings Through Openai Pretrained embeddings

In [ ]:
# get openai api key from platform.openai.com
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or ''

def get_openai_transformer_embeddings(model_name="text-embedding-3-large"):
  return OpenAIEmbeddings(model=model_name,openai_api_key=OPENAI_API_KEY)

embeddings_model = get_openai_transformer_embeddings(model_name="text-embedding-3-large")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


#Now We Will Get Chunks and Pdf Names To Store As a Metadata


In [ ]:
def extract_chunks_and_pdf_names(chunks):
    list_of_chunks = [chunk.page_content for chunk in chunks]
    pdf_names = [chunk.metadata['source'] for chunk in chunks]
    return list_of_chunks, pdf_names

# Example usage:
list_of_Chunks, Pdf_Name = extract_chunks_and_pdf_names(Chunks)

# Printing the total number of chunks and corresponding PDF names
print('The Total Number of Chunks Are', len(list_of_Chunks))
print('PDF Names Are:', len(Pdf_Name))

The Total Number of Chunks Are 1321
PDF Names Are: 1321


# Generate Embeddings And Store Into DataFrame

In [ ]:
def create_embedding_dataframe(embeddings_model, list_of_Chunks, Pdf_Name):
    # Generate embeddings for each chunk
    embeddings = embeddings_model.embed_documents(list_of_Chunks)
    # Generate unique IDs for each chunk
    chunk_ids = [f'vec{i+1}' for i in range(len(list_of_Chunks))]
    # Format chunks with metadata
    formatted_chunks = [{"text": chunk, "Source": pdf_file_name} for chunk, pdf_file_name in zip(list_of_Chunks, Pdf_Name)]
    # Create a DataFrame
    df = pd.DataFrame({
        'id': chunk_ids,
        'values': embeddings,
        'metadata': formatted_chunks
    })
    return df

# Example usage:
# Assuming embeddings_model, list_of_Chunks, and Pdf_Name are already defined
df = create_embedding_dataframe(embeddings_model, list_of_Chunks, Pdf_Name)



#### Generate Vector to Store Data In Pinecone Database

In [ ]:
def create_vectors_from_dataframe(df):
    # Initialize an empty list to store the vectors
    vectors = []
    # Iterate through DataFrame rows
    for _, row in df.iterrows():
        vector = {
            "id": row['id'],
            "values": row['values'],
            "metadata": {"text": row['metadata']['text'], "source": row['metadata']['Source']}
        }
        vectors.append(vector)
    return vectors

result_vectors = create_vectors_from_dataframe(df)




# Intialize Pinecone Database and Fetch index

In [ ]:
def initialize_pinecone(api_key="", environment="", index_name=""):
  pc = Pinecone(api_key=api_key, environment=environment)
  index = pc.Index(index_name)
  return index

pinecone_index = initialize_pinecone()



# Store Vector Data In terms of Batches

In [ ]:
def upsert_vectors_in_batches(index, vectors, vector_length=1000, batch_size=30, namespace="ns1"):
    if len(vectors) < vector_length:
        # If the list is smaller than the batch size, upsert in a single batch
        index.upsert(vectors, namespace=namespace)
    else:
        for batch_start in range(0, len(vectors), batch_size):
            batch_end = min(batch_start + batch_size, len(vectors))
            batch_data = vectors[batch_start:batch_end]
            # Use Pinecone's batch upsert function to insert or update documents in batch
            index.upsert(batch_data, namespace=namespace)

# Assuming 'pinecone_index' is already defined and 'result_vectors' is the list of vectors
upsert_vectors_in_batches(pinecone_index, result_vectors, batch_size=30, namespace="ns1")


#Get Similar Docs From Vector Database


In [ ]:
def get_similiar_docs(index,query, k=5, score=False):
    result=index.query(
    namespace="ns1",
    vector=embeddings_model.embed_query(query),
    top_k=k,
    include_values=True,
    include_metadata=True)
    return result['matches'][0]['metadata']['text']+"\n"+result['matches'][1]['metadata']['text']+"\n"+result['matches'][2]['metadata']['text']+"\n"+result['matches'][3]['metadata']['text']+"\n"+result['matches'][4]['metadata']['text']


# Apply Chatgpt TO Get Refined Results

In [ ]:

# Initialize the OpenAI API key
openai.api_key = ''

def return_answer (context,query):
  # Make a request using the OpenAI API
  response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": '''Fallow all the Following Instrcutions Given Below
        1.You are a helpful AI assistant I am going to Pass You Context and Query inside user content.
        2.Provide the Best, Professional, Error Free, Cleaned and Refined Answer according to User Query Based on Context.
        3.Revise the provided text to eliminate any spacing errors and ensure a professional presentation.
        4.Correct spacing should be applied consistently throughout the text.'''},
        {"role": "user", "content": f"Context: \n{context}\n\nQuery: {query}"}
        ],
    temperature=0,
    max_tokens=3000,
    top_p=0,
    frequency_penalty=0,
    presence_penalty=0)
  return response['choices'][0]['message']['content']


#Calling Get Similarity and Return Answer Function

In [ ]:
def get_answer(query):
  context = get_similiar_docs(pinecone_index,query)
  answer = return_answer (context,query)
  return answer

# Enter Your Query to Get Answer

In [ ]:
Enter_Your_query='Consider postponing the application if'
answer = get_answer(Enter_Your_query)


#Printing Answer

In [ ]:
print(answer)

The application should be considered for postponing if the following conditions apply:

1. The applicant has not met the standard of medical care as outlined below:
   - Ages 60 to 64, must have received doctor’s care within the past 5 years.
   - Ages 65 to 70, must have received doctor’s care within the past 2 years.
   - Ages 71 to 75, must have received doctor’s care within the past year.
   If the application is submitted and formally postponed for not meeting these requirements, the applicant will be required to establish care with a primary care physician with more than one visit and any labs and/or age-appropriate testing completed. Copies of the records should be submitted at no expense to Nationwide at the time of reapplication.

2. The applicant is being evaluated for an undiagnosed medical condition. In this case, it is advisable to postpone the submission until all evaluations have been completed and a diagnosis has been made.

3. The applicant has any outstanding tests, l